# Project (Section 8)

This is where we left off in the previous section project:

In [1]:
from datetime import date
from enum import Enum
from typing import Annotated, TypeVar
from uuid import uuid4
from pydantic import BaseModel, ConfigDict, Field, field_serializer
from pydantic.alias_generators import to_camel
from pydantic import UUID4


class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"


T = TypeVar('T')
BoundedString = Annotated[str, Field(min_length=2, max_length=50)]
BoundedList = Annotated[list[T], Field(min_length=1, max_length=5)]


class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

    id_: UUID4 | None = Field(alias="id", default_factory=uuid4) 
    manufacturer: BoundedString
    series_name: BoundedString
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = False
    manufactured_date: date = Field(validation_alias="completionDate", ge=date(1980, 1, 1))
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD", 
        serialization_alias="baseMSRPUSD"
    )
    top_features: BoundedList[BoundedString] | None = None
    vin: BoundedString
    number_of_doors: int = Field(
        default=4, 
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
    )
    registration_country: BoundedString | None = None
    license_plate: BoundedString | None = None

    @field_serializer("manufactured_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

There are two main changes we are going to make on our model.

First, we want to add an additional field to capture when an automobile was registered. To do so, add a field named `registration_date` that is implemented as follows:
- place it right after `registration_country` in the model
- it should be a `date` object
- it should be optional and default to `None`
- it should deserialize from and serialize to the camel case version of the field name
- it cannot be earlier than the `manufactured_date`
- just like `manufactured_date` it should serialize the data to a `YYYY/MM/DD` format for JSON serialization. (Hint: you do not need to define a second serializer for that field! The syntax is the same as what I showed you when applying the same decorator validator to multiple fields)


Secondly, we want to ensure that the `registration_country` only allows values from a pre-determined list of countries.

We are not going to use an enum for this, as there would simply be too many values. Instead, we are going to validate the country name against a "database".

For this exercise we are not going to use an actual database, instead you can use the dictionary provided below. The dictionary keys are going to become the accepted "input" value for country names, and each key's value contains a tuple consisting of the country name (properly formatted), and the 3 character country code (we won't use the country code right now, but we will later).

Create a custom validator for `registration_country` that validates the data being deserialized is one of the keys in that dictionary, and replace the deserialized value with the country name from the first name in the tuple.

For example, if the input data contains:
```
{
    ...,
    "registrationCountry": "UK",
    ...
}
```
then, since our "database" gives us this info:
```
"uk": ("United Kingdom", "GBR")
```
the deserialized value in our model should become `United Kingdom`. 

Your validator should validate a country name based on the lower-cased and stripped version of the string - i.e. input data such as `"UK"`, `"Uk"`, `"uk "` should all end up being matched with the key `"uk"` in the database.

Use an annotated type to do this - name your new annotated type `Country`.

In [2]:
countries = {
    "australia": ("Australia", "AUS"),
    "canada": ("Canada", "CAN"),
    "china": ("China", "CHN"),
    "france": ("France", "FRA"),
    "germany": ("Germany", "DEU"),
    "india": ("India", "IND"),
    "mexico": ("Mexico", "MEX"),
    "norway": ("Norway", "NOR"),
    "pakistan": ("Pakistan", "PAK"),
    "san marino": ("San Marino", "SMR"),
    "sanmarino": ("San Marino", "SMR"),
    "spain": ("Spain", "ESP"),
    "sweden": ("Sweden", "SWE"),
    "united kingdom": ("United Kingdom", "GBR"),
    "uk": ("United Kingdom", "GBR"),
    "great britain": ("United Kingdom", "GBR"),
    "britain": ("United Kingdom", "GBR"),
    "us": ("United States of America", "USA"),
    "united states": ("United States of America", "USA"),
    "usa": ("United States of America", "USA"),
}

I obtained this list by selecting a small portion of the data available 
here: [https://www.iban.com/country-codes](https://www.iban.com/country-codes).

Feel free to add yourself in the list if you're not already!

Here is a sample data input and serialized output (dict and JSON):

In [3]:
from uuid import UUID

data = {
    "id": "c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7",
    "manufacturer": "BMW",
    "seriesName": "M4 Competition xDrive",
    "type": "Convertible",
    "isElectric": False,
    "completionDate": "2023-01-01",
    "msrpUSD": 93_300,
    "topFeatures": ["6 cylinders", "all-wheel drive", "convertible"],
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": "us",
    "registrationDate": "2023-06-01",
    "licensePlate": "AAA-BBB"
}

expected_by_alias = {
    'id': UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'),
    'manufacturer': 'BMW',
    'seriesName': 'M4 Competition xDrive',
    'type': AutomobileType.convertible,
    'isElectric': False,
    'manufacturedDate': date(2023, 1, 1),
    'baseMSRPUSD': 93300.0,
    'topFeatures': ['6 cylinders', 'all-wheel drive', 'convertible'],
    'vin': '1234567890',
    'numberOfDoors': 2,
    'registrationCountry': 'United States of America',
    'registrationDate': date(2023, 6, 1),
    'licensePlate': 'AAA-BBB'
}

expected_json_by_alias = '{"id":"c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7","manufacturer":"BMW","seriesName":"M4 Competition xDrive","type":"Convertible","isElectric":false,"manufacturedDate":"2023/01/01","baseMSRPUSD":93300.0,"topFeatures":["6 cylinders","all-wheel drive","convertible"],"vin":"1234567890","numberOfDoors":2,"registrationCountry":"United States of America","registrationDate":"2023/06/01","licensePlate":"AAA-BBB"}'

Don't forget to also test with invalid data for registration date and country name!